# MetPy Basic Sounding

## Justin Richling
## 09/20/18


# Plotting of Univeristy of Wyoiming Upper Air Data


Read in raw data for specific time and station (List in Notebook down below)
* Take raw data from columns and plot with clean plotting tools

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import metpy.plots as plots
from metpy.units import units
%matplotlib inline
from siphon.simplewebservice.wyoming import WyomingUpperAir
import numpy as np
import metpy.calc as mcalc
import os

In [ ]:
# Uncommment for archived date
dateCurrent = datetime(2018,10,31,12) 

# Set today's date for search and naming purposes
#now = datetime.now() 
#dateCurrent = datetime(int('{0:%Y}'.format(now)),int('{0:%m}'.format(now)),int('{0:%d}'.format(now)),12)

print dateCurrent

# Make a string for YYYYMMDD for naming saved figure later
date = str(dateCurrent)[0:4]+str(dateCurrent)[5:7]+str(dateCurrent)[8:10]
print date

# Set a path to save the plots with string format for the date to set the month and day 
im_save_path ="/path/to/saved/files/"+'{0:%m_%d}'.format(dateCurrent)+"/"
print im_save_path

# Check to see if the folder already exists, if not create it
if not os.path.isdir(im_save_path):
    os.makedirs(im_save_path)

# Uncomment if you want to automatically change to the map folder    
#os.chdir(im_save_path)

![alt text](http://weather.uwyo.edu/upperair/naconf.jpg)

In [ ]:
StationsList = ["Wallops Island, VA","Upton, NY","Chatham, MA","Albany, NY","Sterling, VA","Gray, ME",\
"Buffalo, NY","Pittsburgh, PA","Wilmington, OH","White Lake, MI","Blacksburg, VA","Greensboro, NC",\
"Newport, NC","Charleston, SC","Jacksonville, FL","Tampa Bay, FL","Miami, FL","Tallahassee, FL",\
"Shelby Cnty. Airport, AL","Peachtree City, GA","Nashville, TN","Gaylord, MI","Green Bay, WI","Davenport, IA",\
"Lincoln, IL","Springfield, MO","Little Rock, AR","Jackson Thomas, MS","Slidell Muni., LA","Lake Charles, LA",\
"Shreveport, LA","Norman, OK","Dodge City, KS","Topeka, KS","Omaha, NE","Chanhassen, MN","International Falls, MN",\
"Aberdeen, SD","Bismarck, ND","Rapid City, SD","North Platte, NE","Amarillo, TX","Midland, TX","Del Rio, TX",\
"Corpus Christi, TX","Brownsville, TX","Ft. Worth, TX","Santa Teresa, NM","Albuquerque, NM","Denver, CO","Grand Juncion, CO",\
"Riverton, WY","Glasgow, MT","Great Falls, MT","Salt Lake City, UT","Flagstaff, AZ","Tuscon, AZ","Yuma Prarie Grnds, AZ","Las Vegas, NV",\
"Elko, NV","Boise, ID","Spokane, WA","Quillayute, WA","Salem, OR","Medford, OR","Reno, NV",\
"Oakland, CA","Vandenberg Air Force Base, CA","San Diego, CA"]

StationNumList = [72402,72501,74494,72518,72403,74389,72528,72520,72426,72632,72318,72317,72305,72208,72206,72210,\
72202,72214,72230,72215,72327,72635,72645,74455,74560,72440,72340,72235,72233,72240,72248,72357,72451,72456,\
72558,72649,72747,72659,72764,72662,72562,72363,72265,72261,72251,72250,72249,72364,72365,72469,72476,72672,\
72768,72776,72572,72376,72274,74004,72388,72582,72681,72786,72797,72694,72597,72489,72493,72393,72293]

StationFinal = dict(zip(StationsList,StationNumList))

In [ ]:
# Lets take a look at Denver, DNR
station = 'DNR'

#station = 'GJT'
#station = raw_input("Which station? ")

In [ ]:
# Pulling a request for U of Wyo's server:
dateCurrent_db = WyomingUpperAir.request_data(dateCurrent,station)

# Print it out:
dateCurrent_db

In [ ]:
# Print out what units say dewpoint are:
dateCurrent_db.units["dewpoint"]

<h2><font><center>-- Dew Point, Ambient Temperatures, U and V Winds  --</center></font></h2>

In [ ]:
p = dateCurrent_db["pressure"].values * units(dateCurrent_db.units["pressure"])
#print "Pressures:",p,"\n"

# We can essentially take a slice of the data as a mask variable
mask = p >= 100 * units.hPa

T = dateCurrent_db["temperature"].values * units(dateCurrent_db.units["temperature"])
Td = dateCurrent_db["dewpoint"].values * units(dateCurrent_db.units["dewpoint"])
u = dateCurrent_db["u_wind"].values * units(dateCurrent_db.units["u_wind"])
v = dateCurrent_db["v_wind"].values * units(dateCurrent_db.units["v_wind"])

<h2><font><center>-- Setting up the logrithmic scale for y-axis (wind barbs)  --</center></font></h2>

In [ ]:
# We can plot on specific intervals for wind barb location:

interval = np.logspace(2,3)*units.hPa
idx = mcalc.resample_nn_1d(p,interval)

<h2><font><center>-- Find Max/Min of Temp and Dew Pt to help Automate Axis Generation  --</center></font></h2>

In [ ]:
# Dew Point
td = dateCurrent_db["dewpoint"].values
t_min = td.min()

# Ambient Temperature
t = dateCurrent_db["temperature"].values
t_max = t.max()

print "Dew pt min:",t_min,"Temp max:",t_max

<h2><font><center>-- Create Plot  --</center></font></h2>

In [ ]:
# Create new figure
fig = plt.figure(figsize=(10,10))

# Create the new skew-T figure
skew = plots.SkewT(fig)

# Plot Temperatures (red) and Dew Pt. (green)
skew.plot(p,T,"red")
skew.plot(p,Td,"green")

# Set extent for both axes
skew.ax.set_ylim(1000,100)
skew.ax.set_xlim(-65,t_max+15)
skew.ax.set_xlim(t_min+30,t_max+15)

#skew.plot_barbs(p2[mask],u[mask],v[mask])
#skew.plot_barbs(p2[mask][::5],u[mask][::5],v[mask][::5])
skew.plot_barbs(p[idx][::2],u[idx][::2],v[idx][::2])

# Thermodynamic lines
skew.plot_dry_adiabats(alpha=0.4)
skew.plot_moist_adiabats(alpha=0.4)
skew.plot_mixing_lines(alpha=0.6)

# Add logos
plots.add_metpy_logo(fig, x=57, y=45)
plots.add_unidata_logo(fig, x=58, y=510)

# Plot title
plt.title(station+"  "+str(dateCurrent)+"Z", fontsize=25)

# Label axes
plt.xlabel(r'Temperature:   ($^{o}$C)', fontsize='large', fontweight='bold')
plt.ylabel(r'Pressure:   (hPa)', fontsize='large', fontweight='bold')

# Save figure to desired location on computer
fig.savefig(im_save_path+"Sounding_"+station+"_"+date+".png",bbox_inches='tight')